<a href="https://colab.research.google.com/github/b202113025/MMFADataset/blob/main/train_Data_xlsx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# 📌 中文廣告真偽二分類 BERT 微調（簡化版）
# 使用 train_Data.xlsx
# 不需要 W&B
# ===============================

# 1️⃣ 安裝必要套件（Colab 專用）
!pip install -q torch transformers scikit-learn jieba openpyxl

# -------------------------------
# 2️⃣ 匯入套件
# -------------------------------
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from google.colab import files

# -------------------------------
# 3️⃣ 上傳 Excel 檔
# -------------------------------
uploaded = files.upload()  # 選擇 train_Data.xlsx 上傳

# 讀取 Excel
df = pd.read_excel("train_Data.xlsx")

# 清理空值
df = df.dropna(subset=['content_cleaned', 'label'])

# 如果 label 是文字，轉成數字
# df['label'] = df['label'].map({'真':1, '假':0})

# -------------------------------
# 4️⃣ 資料切分
# -------------------------------
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df['content_cleaned'], df['label'], test_size=0.3, random_state=42, stratify=df['label']
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

# -------------------------------
# 5️⃣ Tokenizer
# -------------------------------
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

def encode_texts(texts):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

train_encodings = encode_texts(train_texts)
val_encodings = encode_texts(val_texts)
test_encodings = encode_texts(test_texts)

# -------------------------------
# 6️⃣ 建立 Dataset
# -------------------------------
class AdDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

train_dataset = AdDataset(train_encodings, train_labels)
val_dataset = AdDataset(val_encodings, val_labels)
test_dataset = AdDataset(test_encodings, test_labels)

# -------------------------------
# 7️⃣ 模型
# -------------------------------
model = BertForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=2)

# -------------------------------
# 8️⃣ 訓練參數（不用 W&B）
# -------------------------------
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to=[]  # <- 這行取消 W&B
)

# -------------------------------
# 9️⃣ 評估指標
# -------------------------------
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# -------------------------------
# 🔟 Trainer
# -------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# -------------------------------
# 1️⃣1️⃣ 訓練模型
# -------------------------------
trainer.train()

# -------------------------------
# 1️⃣2️⃣ 評估測試集
# -------------------------------
results = trainer.evaluate(test_dataset)
print("測試集結果:", results)

# -------------------------------
# 1️⃣3️⃣ 儲存模型
# -------------------------------
model.save_pretrained("mmfa-bert-model")
tokenizer.save_pretrained("mmfa-bert-tokenizer")

# 自動打包下載
!zip -r mmfa-bert-model.zip mmfa-bert-model
files.download("mmfa-bert-model.zip")

Saving train_Data.xlsx to train_Data.xlsx


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.116094,0.969697,0.982143,0.956522,0.969163
2,No log,0.139765,0.965368,0.981982,0.947826,0.964602
3,No log,0.149868,0.965368,0.981982,0.947826,0.964602


測試集結果: {'eval_loss': 0.16078200936317444, 'eval_accuracy': 0.9482758620689655, 'eval_precision': 0.940677966101695, 'eval_recall': 0.9568965517241379, 'eval_f1': 0.9487179487179487, 'eval_runtime': 1.3968, 'eval_samples_per_second': 166.096, 'eval_steps_per_second': 5.727, 'epoch': 3.0}
  adding: mmfa-bert-model/ (stored 0%)
  adding: mmfa-bert-model/config.json (deflated 55%)
  adding: mmfa-bert-model/model.safetensors (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ===============================
# 📌 中文廣告真假即時預測範例
# 使用 mmfa-bert-model
# ===============================

from transformers import BertTokenizer, BertForSequenceClassification
import torch

# -------------------------------
# 1️⃣ 載入已訓練好的模型
# -------------------------------
model_path = "mmfa-bert-model"  # 如果在 Colab，可以直接上傳 zip 後解壓
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# 設定模型為評估模式
model.eval()

# -------------------------------
# 2️⃣ 預測函數
# -------------------------------
def predict_ad(text):
    # 編碼文字
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    # 模型輸出
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    pred = torch.argmax(logits, dim=1).item()  # 取最大值的索引

    # 返回結果
    return "誤導廣告 (假)" if pred == 1 else "正常廣告 (真)"

# -------------------------------
# 3️⃣ 測試範例
# -------------------------------
sample_ads = [
    "本產品可治百病，包治百病，立即見效！",
    "新鮮水果，天然無添加，健康美味。"
]

for ad in sample_ads:
    result = predict_ad(ad)
    print(f"廣告內容: {ad}\n預測結果: {result}\n")
